In [169]:
from spinbox import *
from itertools import count
seeder = count(0,1) # generates a list of counting numbers for rng "seeds" (not really seeds in Numpy, but indices of seeds)

def npr(name, thing):
    print(name,"\n",thing,"\n"+16*"-")

# Tutorial 2 : Simple calculations in the non-entangled space using product states

The classes `ProductState` and `ProductOperator` are analogous to `HilbertState` and `HilbertOperator` but they are individually restricted to tensor products of one-body states and one-body operators respectively.

To begin, let's do the same basic calculations we did before. A `ProductState` may be instantiated like so.

In [170]:
ket_1 = ProductState(n_particles=2, ketwise=True, isospin=False).randomize(seed=next(seeder))
print(ket_1)

ProductState ket of 2 particles: 
ket #0:
[[ 0.18722951-0.79768503j]
 [-0.19672223+0.53846454j]]
ket #1:
[[0.36861193+0.75055118j]
 [0.06037799+0.54511711j]]



Note that our usual matrix representation is replaced with just the one-body vectors. You can project a `ProductState` into a `HilbertState` by the `.to_full_basis()` method.

In [171]:
print(ket_1.to_full_basis())

HilbertState ket of 2 particles:
[[ 0.66771848-0.15351089j]
 [ 0.4461363 +0.05389939j]
 [-0.47665936+0.05083436j]
 [-0.30540393-0.07472524j]]


Addition and subtraction are not allowed for `Product` objects, because in general doing so will not product another `Product` object. 
We do, however, have scalar multiplication. There are two built-in ways to do it.

For a product state $|s_1\rangle \otimes |s_2\rangle$ multiplied by a scalar $c$,

$$c (|s_1\rangle \otimes |s_2\rangle) = c|s_1\rangle \otimes |s_2\rangle = |s_1\rangle \otimes c |s_2\rangle = c^{1/A}|s_1\rangle \otimes c^{1/A}|s_2\rangle$$

We can either choose a particle to multiply by $c$, or we multiply all of them by $c^{1/A}$.

In [172]:
c = 1000.0

npr("Multiplying the first particle:", ket_1.scale_one(0,c))
npr("Multiplying the second particle:", ket_1.scale_one(1,c))
npr("Multiplying all particles:", ket_1.scale_all(c))

Multiplying the first particle: 
 ProductState ket of 2 particles: 
ket #0:
[[ 187.22951251-797.68503332j]
 [-196.72222747+538.46454152j]]
ket #1:
[[0.36861193+0.75055118j]
 [0.06037799+0.54511711j]]
 
----------------
Multiplying the second particle: 
 ProductState ket of 2 particles: 
ket #0:
[[ 0.18722951-0.79768503j]
 [-0.19672223+0.53846454j]]
ket #1:
[[368.61193301+750.55118201j]
 [ 60.37799402+545.11711023j]]
 
----------------
Multiplying all particles: 
 ProductState ket of 2 particles: 
ket #0:
[[ 5.92071705-25.22501561j]
 [-6.22090305+17.0277439j ]]
ket #1:
[[11.65653281+23.73451236j]
 [ 1.90931982+17.2381166j ]]
 
----------------


Multiplication is handled in the same way as for `HilbertState`.

In [173]:
bra_2 = ProductState(n_particles=2, ketwise=False, isospin=False).randomize(seed=next(seeder))

npr("< state 1 | state 1 > = ", ket_1.dagger() * ket_1)
npr("< state 2 | state 1 > = ", bra_2 * ket_1)
npr("| state 1 > < state 2 | = " , ket_1 * bra_2)

< state 1 | state 1 > =  
 (0.9999999999999999+0j) 
----------------
< state 2 | state 1 > =  
 (0.0976436722152378-0.0992577246900124j) 
----------------
| state 1 > < state 2 | =  
 ProductOperator
Op 0 Re:
[[ 0.58434323  0.37864789]
 [-0.41250184 -0.29851431]]
Op 0 Im:
[[-0.07883252 -0.42462983]
 [ 0.00592682  0.26332491]]
Op 1 Re:
[[ 0.33643132 -0.58753285]
 [ 0.20042615 -0.25350879]]
Op 1 Im:
[[ 0.03210537 -0.48968377]
 [ 0.09468723 -0.43289161]]
 
----------------


Likewise, the outer product has produced a `ProductOperator` object. The `ProductOperator` behaves analogous to the `HilbertOperator`.

In [174]:
operator = ProductOperator(n_particles=2, isospin=False).apply_sigma(particle_index=0, dimension=0) # sigma x on particle 1
print(operator)

ProductOperator
Op 0 Re:
[[0. 1.]
 [1. 0.]]
Op 0 Im:
[[0. 0.]
 [0. 0.]]
Op 1 Re:
[[1. 0.]
 [0. 1.]]
Op 1 Im:
[[0. 0.]
 [0. 0.]]



*Example*: Let's use the `ProductState` and `ProductOperator` classes to show that $\frac{1}{2} (\vec{\sigma}_1 \cdot \vec{\sigma}_2 + 1 )$ constitutes an exchange of the spin DOFs between particles.

This is a good example calculation because we immediately see the difficulty presented by the sum of operations: $\hat{\sigma}_{1x}\hat{\sigma}_{2x} + \hat{\sigma}_{1y}\hat{\sigma}_{2y} + \hat{\sigma}_{1z}\hat{\sigma}_{2z} + I$. We cannot add these operators together and get a `ProductOperator`. The best way to compute this is to break it into individual brackets, then add those together.

$$ \frac{1}{2} \left( 1 + \langle \psi | \hat{\sigma}_{1x}\hat{\sigma}_{2x}| \psi \rangle + \langle \psi |\hat{\sigma}_{1y}\hat{\sigma}_{2y}| \psi \rangle + \langle \psi |\hat{\sigma}_{1z}\hat{\sigma}_{2z}| \psi \rangle \right) =\langle \psi | P_{12} | \psi \rangle$$

The `ProductState` class has a built-in exchange method, which I use here.

In [175]:
# state
psi = ProductState(n_particles=2, ketwise=True, isospin=False).randomize(seed=next(seeder))

# left hand side
lhs = 1.
for dimension in (0,1,2):
    op = ProductOperator(2, isospin=False).apply_sigma(0,dimension).apply_sigma(1,dimension)
    lhs += psi.dagger() * op * psi
lhs *= 0.5
    
# right hand side
rhs = psi.dagger() * psi.exchange(0,1)
npr("< psi | sigma_i . sigma_j + 1 | psi > / 2 = ", lhs)
npr(" < psi | P_ij | psi >  = ", rhs)


< psi | sigma_i . sigma_j + 1 | psi > / 2 =  
 (0.3515775813505698+0j) 
----------------
 < psi | P_ij | psi >  =  
 (0.3515775813505698+0j) 
----------------


We can check this again with spin and isospin DOFs. The full exchange operator is

$$P_{ij} = \frac{1}{4} (\vec{\sigma}_1 \cdot \vec{\sigma}_2 + 1 ) (\vec{\tau}_1 \cdot \vec{\tau}_2 + 1 ) = \frac{1}{4} \left( 1 + \sum_\alpha \sigma_{1\alpha}\sigma_{2\alpha} + \sum_\alpha \tau_{1\alpha}\tau_{2\alpha} + \sum_{\alpha\beta} \sigma_{1\alpha}\tau_{1\beta}\sigma_{2\alpha}\tau_{2\beta} \right) $$ 

In [176]:
# state
psi = ProductState(n_particles=2, ketwise=True, isospin=True).randomize(seed=next(seeder))

# left hand side
lhs = 1.
for a in (0,1,2):
    op = ProductOperator(2, isospin=True).apply_sigma(0,a).apply_sigma(1,a)
    lhs += psi.dagger() * op * psi
for a in (0,1,2):
    op = ProductOperator(2, isospin=True).apply_tau(0,a).apply_tau(1,a)
    lhs += psi.dagger() * op * psi
for a in (0,1,2):
    for b in (0,1,2):
        op = ProductOperator(2, isospin=True).apply_sigma(0,a).apply_sigma(1,a).apply_tau(0,b).apply_tau(1,b)
        lhs += psi.dagger() * op * psi
lhs *= 0.25
    
# right hand side
rhs = psi.dagger() * psi.exchange(0,1)
npr("< psi | (sigma_i . sigma_j + 1) (tau_i . tau_j + 1) | psi > / 4 = ", lhs)
npr(" < psi | P_ij | psi >  = ", rhs)


< psi | (sigma_i . sigma_j + 1) (tau_i . tau_j + 1) | psi > / 4 =  
 (0.08810537032438076-3.751261804793913e-18j) 
----------------
 < psi | P_ij | psi >  =  
 (0.08810537032438082+0j) 
----------------


A `ProductState` can be projected to the full basis by the `.to_full_basis()` method, which just computes the tensor product and returns a `HilbertState`. The same is true for operator classes. The inverse process is not always possible, since a general state is a linear combination of product states. In otherwords if the `HilbertState` has entanglement, then there is no equivalent `ProductState`. However, I have added a few methods to help illuminate these things.

Let's begin with a 3-body product state, then we can entangle the particles by projecting to the full space and applying a two-body operator. 

In [177]:
s_pure = ProductState(n_particles=3, ketwise=True, isospin=False).randomize(seed=next(seeder))
s_entangled = HilbertState(n_particles=3, ketwise=True, isospin=False).randomize(seed=next(seeder))

npr("pure state",s_pure.to_full_basis())
npr("entangled state", s_entangled)

pure state 
 HilbertState ket of 3 particles:
[[ 0.66497576-0.09891649j]
 [-0.00123622-0.63882083j]
 [ 0.11492372+0.16853203j]
 [ 0.17430219-0.08478903j]
 [-0.00768452-0.17080281j]
 [-0.16163923-0.01634436j]
 [ 0.04563134-0.02467863j]
 [-0.01690479-0.0463053j ]] 
----------------
entangled state 
 HilbertState ket of 3 particles:
[[-0.19919328+0.18598239j]
 [-0.32896006+0.40606688j]
 [-0.06169102+0.06775356j]
 [ 0.1044352 -0.30634886j]
 [ 0.28218477-0.23802532j]
 [ 0.02725018+0.3974318j ]
 [-0.13727312+0.05039436j]
 [-0.19493309-0.43024828j]] 
----------------


The `.nearby_product_state()` method does an optimization routine to find a product state that is close to the `HilbertState`. Applying this to the pure state does not give us back the original, however, because this process is not bijective. While states in the full Hilbert space are more general, we often destroy information in the projection from a product state. Note that we get back a new product state, but it's overlap with the original is nearly 1.

In [178]:
s_pure_opt, _ = s_pure.to_full_basis().nearby_product_state(maxiter=1000)
npr("new product state:", s_pure_opt)
npr("overlap:", s_pure_opt.dagger() * s_pure)

[ 0.55848432 -0.12373871  0.80651942  0.46106324  0.30798256  0.05741234 -0.74611729  0.34072421  0.16012347 -0.33362793 -0.73537755 -0.60089136]
Iteration limit reached    (Exit mode 9)
            Current function value: 7.140652194263066e-13
            Iterations: 1000
            Function evaluations: 22498
            Gradient evaluations: 1000
new product state: 
 ProductState ket of 3 particles: 
ket #0:
[[ 0.86471298-0.51858434j]
 [-0.10853206-0.23120539j]]
ket #1:
[[ 0.73996379+0.51092853j]
 [-0.04164558+0.26953191j]]
ket #2:
[[ 0.72538292-0.15528672j]
 [-0.04702299-0.70306566j]]
 
----------------
overlap: 
 (0.9999991549790767+2.2042507541453205e-09j) 
----------------


You can also use the `.nearest_product_state()` method to repeat this process for a list of seeds, which correspond to different initial conditions. This returns the single closest product state found.

In [179]:
seeds = np.arange(10)
s_pure_opt = s_pure.to_full_basis().nearest_product_state(seeds)
npr("new product state:", s_pure_opt)
npr("overlap:", s_pure_opt.dagger() * s_pure)

[ 0.07751917 -0.08144947  0.40361418  0.0661113  -0.59597173  0.4023012   0.80398332  0.58392429 -0.44351574 -0.79750779 -0.69343886  0.0459782 ]
Iteration limit reached    (Exit mode 9)
            Current function value: 2.935799095057073e-12
            Iterations: 100
            Function evaluations: 1805
            Gradient evaluations: 100
[ 0.28995658  0.68936483  0.2321024  -0.91535105  0.78842265  0.38872209 -0.45052154  0.48757733  0.2560794   0.20660169  0.0247513   0.47610108]
Iteration limit reached    (Exit mode 9)
            Current function value: 2.1114827842395726e-13
            Iterations: 100
            Function evaluations: 1832
            Gradient evaluations: 100
[ 0.18777303 -0.51920815 -0.16180312 -0.956359    0.76050528  0.48349204 -0.32321892  0.76856601  0.11015439 -0.21694062  0.41309227 -0.13123239]
Iteration limit reached    (Exit mode 9)
            Current function value: 6.309758410261707e-13
            Iterations: 100
            Function evalu

Doing the same with the entangled state should give us some noticable discrepancy due to entanglement. 

In [180]:
s_entangled_opt = s_entangled.nearest_product_state(seeds)
npr("original entangled state:", s_entangled)
npr("nearby product state:", s_entangled_opt.to_full_basis())
npr("overlap = ", s_entangled_opt.to_full_basis().dagger() * s_entangled)

[ 0.07751917 -0.08144947  0.40361418  0.0661113  -0.59597173  0.4023012   0.80398332  0.58392429 -0.44351574 -0.79750779 -0.69343886  0.0459782 ]
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.011318694673751102
            Iterations: 39
            Function evaluations: 588
            Gradient evaluations: 39
[ 0.28995658  0.68936483  0.2321024  -0.91535105  0.78842265  0.38872209 -0.45052154  0.48757733  0.2560794   0.20660169  0.0247513   0.47610108]
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.011318694673751788
            Iterations: 29
            Function evaluations: 380
            Gradient evaluations: 29
[ 0.18777303 -0.51920815 -0.16180312 -0.956359    0.76050528  0.48349204 -0.32321892  0.76856601  0.11015439 -0.21694062  0.41309227 -0.13123239]
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.011318694673753014
            Iterations: 5